# Chp11. Training Deep Neural Nets

- __발표자 : 정지원__
- __발표일 : 2017. 8. 5(토)__
---

Chp10에서 ANN에 대해 배우고, DNN을 학습시켜봤다. 하지만 겨우 hidden layer 2개정도였는데, 복잡한 문제는 어떻게 해결할까? hidden layer 10겹?

아마 이러한 문제에 봉착할 것이다.

- Vanishing Gradients problem(or exploding gradients problem)

- Large network -> training slow.

- millions of parameters -> overfitting

이번 챕터에서는 이러한 문제를 해결하는 방법 등에 대해 배워본다.

### 1. vanishing/exploding 문제를 해결

### 2. 학습 속도를 올릴 수 있는 방법

### 3. regularization에 대해

## Vanishing/Exploding Gradients Problems

Backpropagation algorithm

앞으로 전파될수록 값이 작아지거나(vanishing gradients)

값이 계속해서 커진다(exploding gradients)

왜 그런가?

sigmoid를 생각해보자.

sigmoid prime의 max값이 1/4이다.

아무리 큰 값이 걸리더라도 계속해서 지수적으로 작아진다.

또한 random gaussian을 따르는 weights/bias를 선택했다고 하면, std가 매우 큰 분포를 따르게 된다.

따라서 network 중 많은 값들이 초기부터 saturated 되고, 학습이 불가능해진다.

![Image](figures/1.png)

위 sigmoid function을 보면, net input=z값의 절댓값이 클수록 saturate 되는 것을 볼 수 있다.(prime 값이 작으므로)

## Xavier and He Initialization

초기값을 바꿔보자.

기본적인 시도는,zeros/random gaussian 이었다.

초기값을 어떻게 주냐에 따라서, vanishing/exploding gradient 문제를 막을 수 있으며, 학습 되는 속도를 높일 수 있다.

![Image](figures/2.png)

예를 들어, Xavier의 경우 위와 같이 초기값을 정한다.

원래의 경우 $\sigma = 1/ \sqrt{n_{inputs}}$ 을 사용하였다.

In [1]:
import numpy as np

for i in range(10):
    print(sum([np.random.normal() for _ in range(100)]))

-11.48963930159386
-1.888146606615182
-10.629263677925056
-19.574903695750336
12.717423813634657
-5.763483749058541
5.175191738380074
-8.493366008455961
2.4653480316193086
12.968833166641954


![Image](figures/3.png)

tf.contrib에 들어있는 fully_connected의 경우, default로 xavier를 사용한다.

## Nonsaturating Activation Functions

ReLU가 등장했다.

적어도 ReLU의 경우 positive values에 대해 saturate되는 문제는 없다.

하지만 ReLU도 negative value에 대해서는 학습이 불완전함을 알 수 있다.

net input(=z)값이 0을 넘지 못하는 경우, 아예 전파를 하지 못하게 된다.(학습이 안 일어난다.)

이를 위해 ReLU의 변형된 함수들이 많이 등장했다.(leaky ReLU 등)

Standard ReLU보다 성능이 항상 좋음이 밝혀졌다.

## $\sigma_{\alpha}(z) = max(\alpha z, z)$

![Image](figures/4.png)

- leaky ReLU -> $\alpha$에 적당한 값을 주는 것이 (ex:0.01)

- RReLU(randomized leaky ReLU) -> $\alpha$ 값을 랜덤하게 조정(일정 범위 안에서)

- PReLU(parametric leaky ReLU) -> 이 녀석도 파라미터처럼 학습

![Image](figures/5.png)

이런 녀석도 있다.(ELU)

이 같은 경우, z<0 인 경우에도 학습이 가능하다.

항상 smooth 하므로, 학습 속도 또한 빨라지게 된다.

하지만 단점으로는, 미분 등을 계산할 때 느리다는 점이다. training을 할 때는 학습이 빠르게 돼서 좋을지 몰라도, 실제 tesing 단계에서는 느리다.(ReLU보다)

### - ELU > leaky ReLU (and its variants) > ReLU > tanh > logistic.

runtime performance가 중요하다면, leaky ReLU > ELU

시간과 컴퓨팅 파워의 여유가 있다면, Cross-validation을 사용하여 여러 Activation Function을 사용해봐라.

특히 네트워크가 overfitting하면 RReLU를, 아주 큰 학습 data의 경우 PReLU를 사용해볼 수 있다.

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

hidden1 = fully_connected(X, n_hidden1, activation_fn=tf.nn.elu)

In [ ]:
def leaky_relu(z, name=None):
    return tf.maximum(0.01 * z, z, name=name)

hidden1 = fully_connected(X, n_hidden1, activation_fn=leaky_relu)

## Batch Normalization

He initialization with ELU(or any variant of ReLU)가 vanishing/exploding을 막아주긴 했지만,

학습되는 동안 또 어떻게 될지는 모르는 일이다.

Batch Normalization(BN)은 이러한 문제(vanishing/exploding)를 해결해주고,

일반적으로, 이전 layers의 parameters가 변경됨에 따라 각 layers의 input distribution이 바뀌는 문제를 해결했다.(Internal Covariate Shift Problem)

이 기법은 각 layers의 activation function 직전에 연산을 추가하고,

입력을 0-centered(mean) 및 정규화 한 다음, layers당 두 개의 새 parameter를 사용하여 결과를 scaling하고 보내는 것으로 구성된다.

(하나는 scaling 용도, 다른 하나는 shifting 용도)

즉, 이 작업을 통해 모델은 각 layers의 inputs의 최적의 scale 및 mean을 학습 할 수 있다.

input들을 중심에 맞추고 정규화하기 위해 알고리즘은 input의 평균 및 표준 편차를 추정해야 한다.

이는 현재의 미니배치에 비해 input의 평균 및 표준 편차를 평가함으로써 바뀌어나간다.

![Image](figures/6.png)

![Image](figures/18.png)

![Image](figures/19.png)

테스트 시간에는 평균과 표준 편차를 계산할 mini-batch가 없으므로, 대신 전체 training set의 평균 및 표준 편차를 사용한다.

이들은 일반적으로 이동 평균을 사용하여 훈련 중에 효율적으로 계산된다.

따라서 각 Batch-normalized 된 layer에 대해 총 4개의 매개 변수가 학습된다.

- ### $\gamma$(scale), $\beta$(offset), $\mu$(mean), $\sigma$(std)

BN을 통해, tanh/logistic을 사용해도 vanishing 되는 문제가 해결됐다.

또한 weight/bias 초기값에도 덜 민감하며, 더 큰 학습률을 사용해도 됐다.

결과적으로 14배 적은 steps으로 같은 Accuracy에 도달할 수 있었다.

심지어 정규화 기능까지...(dropout 등과 같은..)

또한 이렇게 BN된 네트워크를 앙상블로 사용할 수 있다.

그러나 BN은 모델을 복잡하게 한다.(input을 정규화 하지 않아도 되긴하는 장점도 있긴하다.)

또한 Runtime 패널티가 존재한다.(test set에 대해 느리다.)

따라서 예측이 빠르길 원한다면(모바일/IoT 등) BN을 하기 전에, ELU+He 조합부터 도전해봐라.

### Implementing Batch Normalization with TensorFlow

tf는 batch_normalization() 함수를 제공

입력을 표준화하지만 평균과 표준 편차를 직접 계산해야한다.

(학습 도중 미니 배치 데이터 또는 테스트 중 전체 데이터 세트를 기반으로)

parameters를 함수에 전달하면 또한 scaling 및 offset parameter을 생성한다다.

batch_norm() 함수를 사용하면 편하다.

직접 사용하거나 fully_connected() 함수와 다음과 같이 써라.

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import batch_norm

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params = {
    'is_training': is_training,
    'decay': 0.99,
    'updates_collections': None
}

hidden1 = fully_connected(X, n_hidden1, scope="hidden1", normalizer_fn=batch_norm, normalizer_params=bn_params)
hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2", normalizer_fn=batch_norm, normalizer_params=bn_params)
logits = fully_connected(hidden2, n_outputs, activation_fn=None,scope="outputs", normalizer_fn=batch_norm, normalizer_params=bn_params)

is_training을 통해 training/test를 구분한다.

bn_params를 정의합니다. bn_params는 물론 is_training을 포함하여 batch_norm () 함수로 전달 될 매개 변수를 정의하는 사전입니다.

누적 평균을 구하기 위해 deacy 값을 사용한다.

좋은 decay 값은 일반적으로 1에 가깝다. (예 : 0.9, 0.99 또는 0.999).

더 큰 데이터 세트의 경우 9를 더 많이 사용하고 작은 mini-batch를 쓴다.

마지막으로, batch_norm()을 원하면 updates_collections를 None으로 설정해야한다.

(is_training = True 일 때, 일괄 정규화를 수행하기 바로 전에 실행 평균을 업데이트하는 기능)

이 매개 변수를 설정하지 않으면 기본적으로 tf가 실행해야하는 작업 모음에 실행중인 평균을 업데이트하는 작업을 추가한다.

In [ ]:
with tf.contrib.framework.arg_scope([fully_connected], normalizer_fn=batch_norm, normalizer_params=bn_params):
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits = fully_connected(hidden2, n_outputs, scope="outputs", activation_fn=None)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        [...]
        for X_batch, y_batch in zip(X_batches, y_batches):
            sess.run(training_op, feed_dict={is_training: True, X: X_batch, y: y_batch})
            accuracy_score = accuracy.eval(feed_dict={is_training: False, X: X_test_scaled, y: y_test})
            print(accuracy_score)

## Gradient Clipping

Exploding 문제를 막기 위해서, 특정 threshold 값 이상/이하인 경우 Gradient를 Clipping한다.

In [ ]:
threshold = 1.0
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

## Reusing Pretrained Layers

비슷한 task를 하려고 할 때, 다음과 같이 모델을 재사용하기도 한다.

![Image](figures/7.png)

__transfer learning__ 이라고 한다.

학습이 빠를 뿐만 아니라, 데이터 셋이 적게 든다는 장점도 있다.

다만 size가 같지 않으면 resize등을 해야 하므로 비슷한 경우에만 보통 사용한다.

## Reusing a TensorFlow Model

저장하고.. 불러오자.

In [ ]:
[...] # construct the original model
with tf.Session() as sess:
    saver.restore(sess, "./my_original_model.ckpt")
    [...] # Train it on your new task

그러나, 보통 original 모델의 일부만 재사용하고 싶을 것이다.

간단한 방법은, Saver에서 오직 ㄴubset of the variables를 저장하도록 하는 것이다.

다음은 1,2,3 layers만 저장하는 코드다.

In [ ]:
[...] # build new model with the same definition as before for hidden layers 1-3

init = tf.global_variables_initializer()

reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[123]")

reuse_vars_dict = dict([(var.name, var.name) for var in reuse_vars])
original_saver = tf.Saver(reuse_vars_dict) # saver to restore the original model

new_saver = tf.Saver() # saver to save the new model

with tf.Session() as sess:
    sess.run(init)
    original_saver.restore("./my_original_model.ckpt") # restore layers 1 to 3
    [...] # train the new model
    new_saver.save("./my_new_model.ckpt") # save the whole model

먼저 새로운 모델을 만들고, original model의 hidden layers[1:3]을 copy해둔다.

모든 all variables의 node를 초기화 시킨다.

그러면 "trainable=True"를 통해 만든 variables list를 얻을 것이다.

그러고서 hidden[123] (regular expression)을 통해 scope가 맞는 것들만 keep한다.

그러고서 original 모델의 각 variable의 이름과 새로운 모델의 이름을 mapping시킨 dictionary를 만든다.

그러고서 __Saver__를 만들면, 1-3 layers 뿐만 아니라 모든 변수를 포함한 모델이 만들어진다.

여기서 session을 켜고, 초기화를 하게 되면, 1-3이 저장되게 된다.

그러고선 이제 새 모델로 학습 시킨 후 저장하면 된다.

## Reusing Models from Other Frameworks

Theano/Pytorch/caffe 등... 다른 프레임웍에서 가져오고 싶은 경우...

아래 코드는 1st layer에서 가져오는 예시

In [ ]:
original_w = [...] # Load the weights from the other framework
original_b = [...] # Load the biases from the other framework

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
[...] # # Build the rest of the model

# Get a handle on the variables created by fully_connected()
with tf.variable_scope("", default_name="", reuse=True): # root scope
    hidden1_weights = tf.get_variable("hidden1/weights")
    hidden1_biases = tf.get_variable("hidden1/biases")

# Create nodes to assign arbitrary values to the weights and biases
original_weights = tf.placeholder(tf.float32, shape=(n_inputs, n_hidden1))
original_biases = tf.placeholder(tf.float32, shape=(n_hidden1))
assign_hidden1_weights = tf.assign(hidden1_weights, original_weights)
assign_hidden1_biases = tf.assign(hidden1_biases, original_biases)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    sess.run(assign_hidden1_weights, feed_dict={original_weights: original_w})
    sess.run(assign_hidden1_biases, feed_dict={original_biases: original_b})
    [...] # Train the model on your new task

## Freezing the Lower Layers

DNN의 lower level layer들은 이미 low-level features를 찾아냈을 것이다.

따라서 이러한 layers들을 그대로 사용하는 것이 좋다.

Weights를 새로운 DNN을 training하는 동안 __'freeze'__하는 것이 아이디어다.

lower-level layer가 fixed 된다면, high-level layer는 학습이 빠르게 될 것이다.

가장 쉬운 방법은, 학습시킬 변수들을 따로 모아두는 것이다.

In [ ]:
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|outputs")

training_op = optimizer.minimize(loss, var_list=train_vars)

위 예에서는, 3/4 layers를 지정하여, 1/2 layer를 얼려두는 코드다.

## Caching the Frozen Layers

얼려둔 애들을 매번 계산하면 낭비다.

따라서 다음과 같이 얼려둔 layer 마지막 부분의 output을 먼저 계산해낸다.

In [ ]:
hidden2_outputs = sess.run(hidden2, feed_dict={X: X_train})

In [ ]:
import numpy as np

n_epochs = 100
n_batches = 500

for epoch in range(n_epochs):
    shuffled_idx = rnd.permutation(len(hidden2_outputs))
    hidden2_batches = np.array_split(hidden2_outputs[shuffled_idx], n_batches) y_batches = np.array_split(y_train[shuffled_idx], n_batches)
    for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
        sess.run(training_op, feed_dict={hidden2: hidden2_batch, y: y_batch})

이미 계산을 한 후 넣어뒀기 때문에, 더 이상 hidden2까지의 계산은 하지 않게 된다.

## Tweaking, Dropping, or Replacing the Upper Layers

오리지널 모델의 output layer는 보통 새 작업을 할 때는 대체된다. Output 사이즈 부터가 다를 수 있기 때문이다.

마찬가지로, output에 가까운 layers들은 input에 가까운 layers들보다 덜 유용하다. 따라서 얼마나 다시 쓸지를 선택해야 한다.

먼저 모든 layers를 얼려두고, 학습을 시킨 후 성능을 봐라. 그러고서는 상위 hidden(output에 가까운) layers를 하나둘씩 unfreeze한다.

그렇게 되면, performance가 개선됨을 볼 수 있다. training data가 많을수록 더 많이 unfreeze 해봐라.

아직도 좋은 성능이 안나온다면, 아마 training data가 적은 것이다. 그런 경우, 상위 몇 개의 hidden layer를 버린 후에 나머지 hidden layers로 재활용할 적당한 layers 수를 찾게 될 때 까지 도전해봐라.

많은 training data를 가진 경우에는, 상위 hidden layers를 버리지 말고, 새로운 hidden layers를 추가해봐라.

## Model Zoos

사람들이 이미 만들어둔 모델을 살펴보자.

- https://github.com/tensorflow/models

이미 유명한 이미지 분류 네트워크인 VGG, Inception, ResNet 등이 있다.

또 유명한 것이 Caffe's Model Zoo이다.

Many Computer vision models(LeNet, AlexNet, ZFNet, GoogLeNet, VGGNet, inception)

trained on various datasets(ImageNet, Places Database, CIFAR10, etc.)

> Caffe to tensorflow

> - https://github.com/ethereon/caffe-tensorflow

## Unsupervised Pretraining

비슷한 작업에 대한 trained model을 찾을 수 없는 상황이라고 가정해보자. 

또한 labeled training data도 얼마 없는 상황이다.

희망을 잃지 말자!

![Image](figures/8.png)

Unlabeld training data가 충분하다면, RBM과 같은 비지도학습 feature detector 알고리즘을 사용하거나 autoencoders를 사용하여 layer를 하나둘씩 학습시킬 수 있다.

각 layers는 이전에 이미 학습된 layer의 output을 통해 학습된다.

모든 layers가 이렇게 학습이 되면, 기존의 지도학습 방식으로 다시 fine-tune 해준다.

길고 지루하지만 잘 작동한다.

실제로 전에 이러한 기술로 2006년에 Neural Network가 부활했고, Deep Learning이 성공했다.

2010년까지는 RBM을 사용하는 unsupervised pretraining이 표준이었으며, backprop의 등장 이후로 backprop을 이용한 DNN 학습이 더 많이 퍼지게 됐다.

그러나, 복잡한 문제를 풀거나/재사용가능한 유사모델이 없거나/labeled data가 부족한 경우에는 여전히 좋은 선택이다.

## Pretraining on an Auxiliary Task

마지막 옵션은 labled training data를 쉽게 얻거나 생성할 수 있는 보조작업에서 첫 번째 신경망을 학습한 다음 실제 작업을 위해 해당 네트워크의 하위 계층을 재사용하는 것이다.

첫 번째 NN의 lower layers는 두 번째 NN에서 재사용 할 수 있는 feature detector를 배운다.

예를 들어, 얼굴 인식하는 system을 만들려고 하는데, 각 얼굴 사진이 몇 개 없다고 하자. 좋은 분류기를 만들기는 어렵다.

각 개인의 사진 수백장을 모으는 것은 실용적이지 못하다.

그러나 인터넷에서 임의의 사람들의 사진을 많이 수집하여, 두 개의 서로 다른 사진이 동일한 사람의 특징을 이루는지 여부를 감지하는 첫 번째 NN를 학습할 수 있다.

이러한 네트워크는 얼굴에 대한 우수한 feature detector를 학습하므로 하위 layer를 재사용하면 학습 데이터가 거의없는 우수한 얼굴 분류기를 학습할 수 있다.

Unlabeld training 예시를 수집하는 것은 종종 저렴하지만 label을 매기는데 비용이 많이 든다.

이러한 상황에서는 모든 training examples을 "good"이라고 매겨둔 후, 이것들을 손상시켜 많은 새로운 학습 instances를 생성하고

이러한 손상된 instances를 "bad"으로 표시하는 것이 일반적인 테크닉이다.

그런 다음 첫 번째 neuron을 학습시킬 수 있다.

네트워크를 사용하여 인스턴스를 좋거나 나쁘게 분류 할 수 있다.

예를 들어, 수백만 개의 문장을 다운로드하여 "good"이라고 표시 한 다음, 각 문장의 단어를 임의로 변경하고 결과 문장에 "bad"라고 표시 할 수 있다.

NN은 아마 언어에 대해 꽤 많이 알게 되기 때문에,  "The dog sleeps."는 좋은 문장이지만, "The dog they"는 나쁘다고 할 것이다.

lower layer를 재사용하면 많은 language processing 작업에서 도움이 될 것이다.

또 다른 접근 방법은 첫 번째 network를 학습하여 각 학습 instance에 대한 점수를 출력하고,

좋은 instance의 점수가 나쁜 인스턴스의 점수보다 적어도 어느 정도 더 큰지 확인하는 Cost function를 사용하는 것이다.

이것을 _max margin learning_이라고 한다.

## Faster Optimizers

매우 깊은 NN를 학습하는 것은 고통스럽게 느려질 수 있다.

지금까지는 학습 속도에 대해 좋은 initialization 전략을 적용하고, 좋은 activation function을 사용하고, Batch Normalization를 사용하고, 사전 학습된 network의 일부를 재사용하는 등 네 가지 방법으로 교육의 속도를 높이고 더 나은 결과를 얻었다.

또 다른 큰 속도 향상은 일반적인 Gradient Descent Optimizer보다 더 빠른 Optimizer 사용으로 얻게 된다.

- Momentum optimization

- Nesterov Accelerated Gradient

- AdaGrad

- RMSProp

- Adam Optimizer

에 대해 배운다.

> Spoiler alert : 이 섹션의 결론은 거의 항상 Adam optimization을 사용해야 한다는 것이다.

> 따라서 작동 원리에 신경 쓰지 않는다면 GradientDescentOptimizer를 AdamOptimizer로 바꾸고 다음 섹션으로 건너 뛰어라!

> 이 작은 변화만으로도 교육은 일반적으로 몇 배 더 빠르다.

> 그러나 Adam 최적화에는 조정할 수 있는 세 가지 하이퍼파라미터(학습 속도 포함)가 있다.

> default value로도 정상적으로 작동하지만, 만약 조정할 필요가 있다면, 알고리즘을 아는 것이 도움이 될 것이다.

> Adam 최적화는 다른 최적화 알고리즘의 여러 아이디어를 결합하므로 먼저 이러한 알고리즘을 살펴 보는 것이 좋다.

> http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html

![Image](figures/1.gif)

![Image](figures/2.gif)

![Image](figures/3.gif)

## Momentum optimization

기존 Gradient Descent는 학습률 η에 J(θ)의 Gradient를 직접 빼서 가중치 θ를 간단히 업데이트한다.

### $\theta \leftarrow \theta - \eta\nabla_{\theta}J(\theta)$

이전 Gradient에 신경쓰지 않는다. Local Gradient가 작으면 계속 느리게 진행.

![Image](figures/9.png)

모멘텀방식은 이전 Gradient를 고려하여 계산된다. Momentum vector __m__에 학습률 η을 곱한 Local Gradient를 추가하고, 이 Momentum vector를 단순히 빼서 가중치를 업데이트한다.

다시 말해서, Gradient는 속도가 아니라 가속도로 사용된다.

운동량이 너무 커지지 않도록 하기위해 이 알고리즘은 0(고 마찰)과 1(마찰 없음) 사이에서 설정해야 하는 Momentum이라고 하는 새로운 Hyper-parameter β를 도입한다.

일반적인 값은 0.9를 사용

Gradient가 일정하게 유지되는 경우, 마지막 값(즉, 가중치 업데이트의 최대 크기)이 학습 속도 η에 $\frac{1}{1-\beta}$을 곱한 배율과 같다.

예를 들어, β = 0.9 인 경우 속도 1-β는 학습 속도의 10배와 동일하므로 Momentum 방식이 Gradient Descent보다 10배 빠르다.

따라서, 훨씬 빠르게 plateaus에서 벗어날 수 있다.

Input이 매우 다른 스케일을 가질 때 Cost function은 길쭉한 Bowl처럼 보일 것이라 했었다.(Chp4)

![Image](figures/22.png)

Gradient Descent는 가파른 경사면에서 아주 빨리 내려가지만 계곡을 내려가려면 매우 오랜 시간이 걸린다.

반대로 Momentum Optimization은 계곡의 바닥이 최적에 도달할 때까지 계곡의 바닥을 더 빠르게 굴러간다.

BN을 사용하지 않는 DNN에서 상위 layer는 종종 아주 다른 스케일의 입력을 갖기 때문에 Momentum optimization을 사용하면 많은 도움이 된다.

또한 이전의 Local optima를 롤백하는 것을 도울 수도 있다.

> Momentum으로 인해 Optimizer는 약간 overshoot하고, 다시 돌아오고, 다시 overshoot하고, 최소값에 안정적으로 도달하기 전에 여러 번 진동할 수 있다.

> 이것이 시스템에서 약간의 friction을 갖는 이유 중 하나다. 진동을 없애고 수렴 속도를 높여준다.

In [ ]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

한 가지 단점은 튜닝할 또 다른 Hyper-parameter를 추가한다는 것이다.

그러나, Momentum 값 0.9은 일반적으로 잘 작동하며 거의 항상 그라데이션 하강보다 빠르다.

## Nesterov Accelerated Gradient

1983년, Yurii Nesterov가 제안한 Momentum optimization의 작은 변형은 Vanilla Momentum optimization보다 항상 빠르다.

Nesterov Momentum Optimization 또는 Nesterov Accelerated Gradient(NST)는 비용 함수의 기울기를 로컬 위치가 아니라 운동량 방향으로 약간 앞당겨 측정하는 것이다.

vanilla와의 유일한 차이점은 기울기가 θ가 아닌 θ+βm에서 측정된다는 것이다.

![Image](figures/10.png)

이 작은 미세 조정은 일반적으로 운동량 벡터가 올바른 방향(즉, 최적 방향)을 가리키기 때문에 효과가 있다.

Gradient를 사용하는 대신 그 방향으로 조금 더 멀리 측정한 Gradient를 사용하는 것이 약간 정확하기 때문이다.

∇1은 시작점 θ에서 측정된 Cost function의 기울기 / ∇2는 θ + βm에 있는 점에서의 기울기

아래 그림과 같이 최적에 약간 가까운걸 알 수 있다.

![Image](figures/11.png)

이러한 작은 개선 사항이 추가되어, NAG는 일반적인 Momentum Optimization보다 훨씬 빠르다.

또한, 운동량이 계곡을 가로 질러 가중치를 밀면, ▽1은 계곡을 가로질러 더 밀고, ∇2는 Valley의 바닥쪽으로 뒤로 밀어 넣는다.

이렇게하면 진동을 줄이고 더 빠르게 수렴한다.

NAG는 Regular Momentum Optimizer에 비해 거의 항상 학습속도를 향상시킨다.

MomentumOptimizer를 만들 때 use_nesterov = True로 설정하면 된다.

In [ ]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True)

## AdaGrad

Gradient Descent는 가파른 경사면을 빠르게 따라가면서 시작하여 계곡의 바닥을 천천히 내려간다.

알고리즘이 초기에 이를 감지하고 방향을 수정하여 global optimum을 좀 더 향하게 하면 좋을 것이다.

![Image](figures/20.png)

첫 번째 단계는 그래디언트의 제곱을 벡터 s에 쌓는다.(⊗ 기호는 요소 별 곱하기를 나타냄)

이 벡터화된 형식은 벡터 $s$의 각 요소 $s_{i}$에 대해 $s_i \leftarrow s_{i} + (\delta / \delta_{i} J(\theta))^{2}$를 계산하는 것과 같다.

다시 말해서 각 $s_{i}$는 매개 변수 $\theta_{i}$에 대한 Cost function의 편미분의 제곱을 누적한다.

Cost function이 i번째 차원에서 가파른 경우 $s_i$는 반복할 때마다 커질 것이다.

두 번째 단계는 Gradient Descent와 거의 동일하지만 큰 차이점이 있다.

Gradient Descent는 factor $\sqrt{s+\epsilon}$로 축소된다.

⊘ 기호는 요소별 분할(나누기)을 나타내고 ε은 0으로 나누는 것을 방지, 일반적으로 e-10으로 설정

이 벡터화된 형식은 모든 매개 변수 $\theta_i$에 대해 $\theta_{i} \leftarrow \theta_{i} - \eta \partial / \partial \theta_{i} J(\theta) / \sqrt{s_{i} + \epsilon}$를 계산하는 것과 같다.

간단히 말해서, 이 알고리즘은 학습 속도를 떨어뜨리지만 가파른 차원의 경우 더 빠르다.

이를 _adaptive learning rate_라고 한다.

Global Optimum에 보다 더 잘 다가갈 수 있다.

또 다른 이점은 학습 속도 hyper-parameter $\eta$의 조정이 훨씬 쉽다는 것이다.

![Image](figures/12.png)

AdaGrad는 종종 간단한 이차 문제를 잘 처리하지만, 불행히도 학습할 때 너무 일찍 종료하는 경우가 많다.

학습 속도가 너무 낮아져 알고리즘이 전역 최적에 도달하기 전에 완전히 멈춘다. 

따라서 TensorFlow에 AdagradOptimizer가 있더라도 깊은 NN를 학습하는데 사용해서는 안된다. 

선형 회귀와 같은 간단한 작업에서는 효율적일 수 있다.

## RMSProp

AdaGrad가 너무 느려 속도가 빠르지만 Global Optimum으로 수렴하지 못하는 경우에도 RMSProp 알고리즘은 가장 최근의 Gradient에 집중하며 축적함으로써 이를 수정한다.

이것은 첫 번째 단계에서 Exponential decay를 사용하여 수행한다.

![Image](figures/13.png)

Decay rate β는 전형적으로 0.9로 설정된다.

새로운 하이퍼 파라미터다.

그러나, 이 default value(0.9)은 잘 작동하기 때문에, 조정할 필요가 전혀 없을 수도 있다.

In [ ]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.9,
                                      decay=0.9, epsilon=1e-10)

매우 간단한 문제를 제외하고, 이 optimizer는 거의 항상 AdaGrad보다 훨씬 뛰어나다.

또한 일반적으로 Momentum optimization 및 Nesterov Accelerated Gradients보다 뛰어나다.

실제로 Adam 최적화가 이루어질 때까지 많은 연구자가 선호하는 최적화 알고리즘이었다.

## Adam Optimization

> https://arxiv.org/pdf/1412.6980v8.pdf

Adam은 Adaptive moment estimation의 약어이며, Momentum과 RMSProp 방식의 조합이다.

Momentum optimization과 같이, 지수적으로 decaying average된 과거의 Gradient를 기록한다.

또한 RMSProp도 지수적으로 decaying average된 과거의 squared gradient를 기록한다.

이 두 방식을 따른다.

![Image](figures/14.png)

(여기서 T는 iteration number를 나타낸다.)

먼저, 1/2/5 단계만 봐도, Momentum/RMSProp과 매우 유사함을 알 수 있다.

다른 점은, step 1에서 지수적인 decaying sum이 아닌, decaying average를 이용한다는 것이다.

그러나 실제로 상수 factor나 다름 없다.(decaying average는 단지 deacying sum에 $ 1-\beta_{1}$이 곱해졌을 뿐이다.)

step 3,4는 다소 기술적 세부 사항이다.

m, s는 0에서 초기화되기 때문에 교육 시작시 0을 향해 편향되므로 이 두 단계는 교육 시작시 m 및 s를 향상시키는데 도움을 준다.

Momentum deacy hyperparameter $\beta_{1}$은 일반적으로 0.9로 초기화 하며,

Scaling decay hyperparameter인 $\beta_{2}$은 0.999로 초기화 한다.

Smoothing term $\epsilon$는 1e-8과 같은 작은 값으로 초기화한다.

tensorflow의 AdamOptimizer의 default 값이 이렇다.

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

사실 Adam이 adaptive learning rate algorithm이기 때문에, 학습률인 $\eta$의 튜닝이 덜 필요하다.

default value로 $\eta$=0.001을 사용하면 되며, 일반적인 Gradient Descent보다 편하다.

> ### Hessians

> 보통의 Optimization은 Jacobians에 대해서만 의존하나, 2차 편미분인 Hessian을 고려할 수도 있다.

> 하지만 기본적으로 DNN이 수 만개의 parameter를 가지는데, Hessian은 하나의 출력 당 n개의 parameter를 가지므로 계산량이 많아져 사용하진 않는다.

> ### Training Sparse Models

> 방금 제시한 모든 최적화 알고리즘은 밀도가 높은 모델을 생성하므로 대부분의 매개 변수가 0이 아니다. Runtime에 놀랄만큼 빠른 모델이 필요하거나 메모리를 적게 사용해야하는 경우 Sparse 모델로 끝내는 것이 좋습니다.

> 이를 위해, 모델을 평소와 같이 학습시킨 다음 작은 가중치를 없애는 방법을 사용한다.(0으로 바꾼다.)

> 또 다른 옵션은 학습 도중 강력한 $L_{1}$ 정규화를 적용하는 것이다. 이는 최적화 알고리즘에서 가능한 많은 가중치를 0으로 Push하기 때문입니다.(Lasso)

> 마지막으로 Yurii Nesterov가 제안한 기법인 FTRL(Follow The Regularized Leader)라고 불리는 Dual Everaging를 적용하는 방법이 있다. $L_{1}$ 정규화와 함께 사용되는 경우, 이 기술은 매우 Sparse한 모델을 만들어준다. tensorFlow상에서, FTRL-Proximal18이라는 FTRL 변형을 FTRLOptimizer 클래스에 구현했다.

## Learning Rate Scheduling

좋은 학습률을 찾는 것이 까다로울 수 있다.

너무 높게 설정하면 발산

너무 낮게 설정하면 결국에는 최적으로 수렴되지만 매우 오랜 시간이 걸린다.

약간 높게 설정하면 처음에는 빠르게 진행되지만, 최적값 근처에서 춤을 추며 끝나지 않는다.

AdaGrad, RMSProp 또는 Adam과 같은 Adaptive learning rate 최적화 알고리즘을 사용하지 않는다면, 정착하는데 시간이 걸릴 수도 있다.

컴퓨팅 예산이 제한적이라면 제대로 수렴하기 전에 학습을 중단해야 할 수도 있다.

다양한 학습률을 사용하여 곡선을 비교하는 몇 Epoch동안 네트워크를 훈련하면 상당히 좋은 학습 속도를 찾아낼 수 있다.

이상적인 학습 속도는 빠르게 학습되며 좋은 solution으로 수렴 할 것이다.

![Image](figures/15.png)

그러나, Constant learning rate보다 더 잘할 수 있다.

높은 학습 속도로 시작한 다음, 빠른 학습이 중단되면 학습 속도를 줄이면서 최적의 Constant learning rate보다 빠른 속도로 도달 할 수 있다.

학습 중 학습률을 줄이기 위한 여러 가지 전략이 있다.

이러한 전략을 _learning schedules_이라고 부른다.

- Predetermined piecewise constant learning rate

> 처음에는 0.1, 50 epoch 후에는 0.001... 좋기도 하지만, 잘 사용해야 한다.

- Performance scheduling

> N단계마다 Validation error를 통해 개선을 확인하여, 개선이 안되는 경우 $\lambda$만큼 줄인다.

- Exponential scheduling

> 학습률을 함수로 정의, $\eta(t) = \eta_{0} 10^{-t/r}$

> r와 $\eta$값을 tuning해야한다는 단점이 있다. 결국 r step마다 10배씩 줄어든다.

- Power scheduling

> $\eta(t) = \eta_{0} (1+t/r)^{-c}$

> c는 보통 1이며, 위 방법과 비슷하나 더 느리게 줄어든다.


Andrew Senior 외의 2013년 논문에서 Momentum optimization를 사용하여 음성 인식을 위한  DNN을 학습할 때 가장 인기있는 Learning schedules의 성능을 비교했다.

이 설정에서 Performance scheduling과 Exponential scheduling이 모두 잘 수행되었지만, 구현이 더 간단하고 조정하기 쉬운 Exponential scheduling을 선호한다고 결론지었다.

In [ ]:
initial_learning_rate = 0.1
decay_steps = 10000
decay_rate = 1/10
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
training_op = optimizer.minimize(loss, global_step=global_step)

AdaGrad, RMSProp 및 Adam 최적화는 교육중 Learning rate를 자동으로 줄이므로 추가적인 learning schedules를 추가할 필요가 없다.

다른 최적화 알고리즘의 경우 Exponential decay 또는 Performance Scheduling을 사용하면 수렴 속도가 상당히 빨라질 수 있다.

## Avoiding Over tting Through Regularization

#### With four parameters I can fit an elephant and with five I can make him wiggle his trunk.
— John von Neumann, _cited by Enrico Fermi in Nature 427_

DNN는 수백만개의 parameter를 갖는다.

parameter가 너무 많아서 네트워크는 매우 다양한 종류의 복잡한 데이터에 적합한다.

그러나, 이러한 큰 유연성은 또한 training set에 overfitting되는 경향이 있음을 의미한다.

수백만 개의 매개 변수로 전체 동물원에 적합 할 수 있습니다.

N에 가장 널리 사용되는 정규화 기법 중 일부를 소개, tensorflow 구현

- Early Stopping

- l1 and l2 Normailzation

- Drop out

- Max-Normailzation

- Data Augmentation

## Early Stopping

Validation set의 performance가 떨어질 때 중지

tensorflow에서는 정기적(ex:50 steps)으로 값을 저장하여 값이 개선되는지를 확인한다.

가장 좋았던 값을 저장하고, 계속하여 개선이 안되는 경우(2000 steps) 그 저장된 값으로 돌아간다.

Early Stopping은 잘 작동하지만, 다른 정규화 기술과 결합하여 더 좋은 성능을 얻을 수 있다.

## l1 and l2 Regularization

L1, L2 이미 배웠다.

Cost function에 적절한 정규화 term을 간단히 추가

예를 들어, weight1인 hidden layer가 있고, weights2가 가중치인 output layer가 있다고 가정하면 다음과 같이 l1 정규화를 적용 할 수 있다.

In [ ]:
[...] # construct the neural network
base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
reg_losses = tf.reduce_sum(tf.abs(weights1)) + tf.reduce_sum(tf.abs(weights2))
loss = tf.add(base_loss, scale * reg_losses, name="loss")

그러나 많은 layer가 있는 경우, 이 방법은 편리하지 않다.

다행히 tensorFlow에서 더 나은 방법을 제공한다.

변수를 만드는 많은 함수(ex:get_variable() 또는 fully_connected())는 생성된 각 변수 (ex:weights_regularizer)에 \*_regularizer 인수를 허용한다.

weight를 인수로 취하는 모든 함수를 전달할 수 있으며 해당 정규화 Loss을 리턴합니다.

l1_regularizer(), l2_regularizer() 및 l1_l2_regularizer() 함수는 이러한 함수를 반환한다.

In [ ]:
with arg_scope([fully_connected],
               weights_regularizer=tf.contrib.layers.l1_regularizer(scale=0.01)):
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits = fully_connected(hidden2, n_outputs, activation_fn=None,scope="out")

## Dropout

가장 널리 사용되는 방법

최첨단 NN에서도 1-2%의 정확도 향상을 얻는다.

별거 아닌 것 같지만, 95%정도의 Accuracy 상태라면 2%가 40%가 될 수 있다.

매우 간단한 알고리즘

모든 학습 단계에서 모든 뉴런(입력 뉴런을 제외하고 출력 뉴런 제외)은 일시적으로 "Dropped out"되는 확률 p를 가지며, 이는 학습 단계에서 완전히 무시됨을 의미한다.

Hyper parameter p는 Drop-out rate라고하며, 일반적으로 50%로 설정

학습 후, Dropped out하지 않는다.

![Image](figures/16.png)

회사에 50%가 동전을 던져서 나오지 않는다면?

개개인의 능력을 잘 분산시켜야 된다.

따라서 input의 사소한 변화에 덜 민감하게 된다.

결국 더 견고한 네트워크를 얻게 된다.

10,000번을 돌리면, 10,000개의 다른 네트워크가 학습된다.

이로서 앙상블 효과를 얻는다.

작지만 중요한 기술적 세부사항이 하나 있다.

p = 50%이라고 가정하면 테스트 중에 뉴런은 훈련 중 (평균적으로)입력 뉴런의 2배를 받는다.

이 사실을 보완하기 위해 학습 후 각 뉴런의 입력 연결 가중치를 0.5로 곱해야한다.

그렇지 않은 경우, 각 뉴런은 네트워크가 학습된 것보다 약 2배 큰 총 입력값을 얻게 될 것이므로 성능이 좋지 않을 것이다.

결론적으로, 각 입력 weights에 훈련 후 유지확률(1-p)을 곱해야한다.

혹은 다른 방법으로는 각 뉴런의 출력을 학습 중 1-p로 나누면 된다.

이 대안들은 완벽하게는 아니지만 거의 똑같이 잘 작동한다.

tensorflow에서는...

학습 중, 이 기능은 임의로 항목을 떨어뜨리고(0으로 설정) 나머지 항목을 유지 확률로 나눈다.

학습 후에 이 함수는 아무 것도하지 않는다.

In [ ]:
from tensorflow.contrib.layers import dropout

[...]
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')

keep_prob = 0.5
X_drop = dropout(X, keep_prob, is_training=is_training)

hidden1 = fully_connected(X_drop, n_hidden1, scope="hidden1")
hidden1_drop = dropout(hidden1, keep_prob, is_training=is_training)

hidden2 = fully_connected(hidden1_drop, n_hidden2, scope="hidden2")
hidden2_drop = dropout(hidden2, keep_prob, is_training=is_training)

logits = fully_connected(hidden2_drop, n_outputs, activation_fn=None, scope="outputs")

BN을 수행한 것처럼 학습할 때 is_training을 True로 설정하고 테스트 때 False로 설정

모델이 오버피팅 상태인 것으로 확인되면 Drop out rate를 높여보면 된다.

반대로 모델이 Training set에 적합하지 않은 경우 드롭 아웃 비율을 줄여야 한다.

또한 대형 레이어의 드롭 아웃 속도를 높이고 작은 레이어의 드롭 아웃 속도를 줄일 수도 있다.

Drop out은 수렴을 상당히 늦추는 경향이 있지만, 일반적으로 잘 조정하면 모델에 도움이 된다.

따라서, 일반적으로 시간과 노력을 들일만한 가치가 있다.

## Max-Norm Regularization

Max-norm Regularization

각 뉴런에 대해 입력 연결의 가중치 w를 제한하여 ${∥w∥}_{2} ≤ r$, 여기서 $r$은 Hyper-parameter이고, ∥ • ∥2는 l2 Norm Term이다.

일반적으로 각 훈련 단계 후에 ${∥w∥}_{2}$를 계산하고, 필요하다면 w를 Clipping하여 제약 조건을 구현한다.

### $w \leftarrow w \frac{r}{∥w∥_{2}}$

r을 줄이면 정규화 양이 늘어나고 overfitting을 줄일 수 있다.

BN을 사용하지 않는 경우라면, Max-Norm은 Vanishing/Exploding Gradient 문제를 완화하는 데 도움이 될 수 있다.

In [ ]:
threshold = 1.0
clipped_weights = tf.clip_by_norm(weights, clip_norm=threshold, axes=1)
clip_weights = tf.assign(weights, clipped_weights)

In [ ]:
with tf.Session() as sess:
    [...]
    for epoch in range(n_epochs):
        [...]
        for X_batch, y_batch in zip(X_batches, y_batches):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            clip_weights.eval()

In [ ]:
hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
with tf.variable_scope("hidden1", reuse=True):
    weights1 = tf.get_variable("weights")

In [ ]:
hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
[...]

with tf.variable_scope("", default_name="", reuse=True): # root scope
    weights1 = tf.get_variable("hidden1/weights")
    weights2 = tf.get_variable("hidden2/weights")

변수이름을 모르는 경우 TensorBoard를 사용하여 global_variables() 함수를 통해 모든 변수 이름을 출력할 수 있다.

In [ ]:
for variable in tf.global_variables():
    print(variable.name)

위 코드는 잘 작동하지만, 약간 지저분하다.

max_norm_regularizer()함수를 생성하고 이전 l1_regularizer() 함수처럼 사용하는 것이 더 깔끔하다.

In [ ]:
def max_norm_regularizer(threshold, axes=1, name="max_norm", collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # there is no regularization loss term
    return max_norm

max_norm_reg = max_norm_regularizer(threshold=1.0)
hidden1 = fully_connected(X, n_hidden1, scope="hidden1", weights_regularizer=max_norm_reg)

max-norm 정규화는 전체 Cost function에 정규화 손실항을 추가할 필요가 없으므로 max_norm()함수는 __None__을 반환한다.

그러나 각 Training step 후에도 여전히 clip_weights 작업을 실행할 수 있어야하므로 이를 다룰 줄 알아야 한다.

max_norm()함수가 clip_weights 노드를 Max-norm Clipping 연산의 집합에 추가하는 이유다.

이러한 클리핑 작업을 가져와 각 학습단계 후에 실행해야한다.

In [ ]:
clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
    [...]
    for epoch in range(n_epochs):
        [...]
        for X_batch, y_batch in zip(X_batches, y_batches):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)

## Data Augmentation

마지막 정규화 기법인 Data Augmentation은 기존 학습 인스턴스를 새롭게 생성하여 인위적으로 학습 Set의 크기를 늘리는 것이다.

이 방법으로 Overfitting을 줄일 수 있다.

실제 데이터와 같은 instances를 생성해야 한다.

이상적으로 인간은 생성된 instances와 그렇지 않은 것을 알 수 없어야 한다.

또한 단순히 화이트 노이즈를 추가하는 것은 도움이 되지 않는다.

적용한 수정 사항을 학습할 수 있어야 한다.(백색 잡음이 아니다)

예를 들어, 모델이 버섯 사진을 분류하기위한 것이라면, 다양한 양의 Training set에서 약간 이동, 회전 및 크기 조정한 것을 Training set에 추가하면 된다.

이렇게하면 그림에서의 버섯의 위치, 방향, 크기에 덜 민감한 모델이 된다.

조명 조건에 대한 모델의 내성을 높이려면 다양한 대비를 사용하여 많은 이미지를 생성하면 된다.

버섯이 대칭이라면 그림을 수평으로 뒤집을 수도 있다.

이러한 변환을 결합하면 교육 세트의 크기를 크게 늘릴 수 있다.

![Image](figures/17.png)

저장 공간과 네트워크 대역폭을 낭비하기보다는 교육 중에 즉석에서 교육 인스턴스를 생성하는 것이 좋습니다.

TensorFlow는 transposing(shifting), rotating, resizing, flipping, cropping과 밝기, 대비, 채도, 색조 조정과 같은 몇 가지 이미지 조작 작업을 제공한다.

따라서 Image data set에 대한 데이터 확장을 쉽게 구현할 수 있다.

## Practical Guidelines

![Image](figures/21.png)


- 좋은 학습률을 찾을 수 없는 경우 (수렴 속도가 너무 느려서 학습 속도를 높이고 수렴이 빠르지만 네트워크의 정확도가 좋지않은 경우) exponential과 같은 learning schedule을 추가할 수 있다.

- Training set이 너무 작으면 Augmentation을 통해 극복한다.

- Sparse 모델이 필요한 경우 L1 정규화를 추가할 수 있다. 선택적으로 학습 후 작은 가중치를 0으로 만들어 봐라. 더욱 Sparse한 모델이 필요하면 L1 정규화와 함께 Adam 대신 FTRL을 사용해보자.

- Runtime이 아주 빠른 모델이 필요한 경우, BN를 하지말고 ELU activation function을 사용해라. Sparse 모델을 사용하면 좋다.

이 가이드 라인에 따라, Deep Neural Network을 학습할 준비가 됐다.

1-machine을 사용하는 경우 학습을 완료하는데 며칠 또는 몇 달 동안 기다려야 할 수도 있다.

다음 장에서는 tensorFlow를 사용하여 여러 서버와 GPU에서 모델을 학습하고 실행하는 방법에 대해 설명한다.